In [1]:
#### Se importan todas las librerías necesarias ####
import plotly as px                         # Cosas de plotly generales. Como los colores
import plotly.graph_objs as go              # Para generar las figuras y los charts
import plotly.io as pio                     # Interfaz para guardar en SVG
import pandas as pd                         # Para leer el CSV con los datos
import numpy as np

In [2]:
# You can also plot your graphs offline inside a Jupyter Notebook Environment. First you need to initiate the Plotly Notebook mode as below:
#init_notebook_mode(connected=True) # Hola

In [3]:
#### Se cargan los datos en CSV con la librería Pandas ####
data1 = pd.read_csv("data.csv",
                    sep=';',        # Por defecto es ,
                    decimal=",",     # Por defecto es .
                    )
#print(data1) # Muestra todas las líneas en texto feo mal colocado
data1.head() # Muestra las 5 primeras pero bien colocadas, hasta la columna de los índices

,Label,ClassifierType,BalancerType,Setting,Accuracy,Fmeasure
0,Complete SVM,SVM,Imbalanced,Complete,81.0850,0.000000
1,Complete SVM,SVM,random undersampling,Complete,75.5450,57.756910
2,Complete SVM,SVM,Smote,Complete,77.9549,56.994479
3,Complete Tree,Tree,Imbalanced,Complete,78.3949,37.722132
4,Complete Tree,Tree,random undersampling,Complete,74.0206,51.432762


In [4]:
# function below sets the color based on amount
def SetMarkerForm(x):
    Symbol1=0 # Imbalanced
    Symbol2=5 # Random
    Symbol3=2 # SMOTE

    if(x=="Imbalanced"):
        return Symbol1
    elif(x=="random undersampling"):
        return Symbol2
    else:
        return Symbol3

# function below sets the color based on amount
def SetMarkerColor(x):
    color1=px.colors.qualitative.T10[0] # Complete - blue
    color2=px.colors.qualitative.T10[2] # No cleaning - red
    color3=px.colors.qualitative.T10[1] # No filtering - orange
    color4=px.colors.qualitative.T10[3] # No segment - celeste
    color5=px.colors.qualitative.T10[4] # Reduced inputs - green

    if(x=="Complete"):
        return color1
    elif(x=="No cleaning"):
        return color2
    elif(x=="No filtering"):
        return color3
    elif(x=="No segment"):
        return color4
    else:
        return color5

# function below sets the color based on amount
def SetBorderMarkerColor(x):
    borderColorSVM  = px.colors.qualitative.T10[7]    # Pastel pink
    borderColorTree = px.colors.qualitative.Dark24[5] # Black
    if(x=="SVM"):
        return borderColorSVM
    elif(x=="Tree"):
        return borderColorTree
    else:
        return borderColorTree

# Frente de Pareto
A = data1[(data1.BalancerType == "random undersampling") & (data1.Setting == "Minimum inputs") & (data1.ClassifierType == "SVM")]
B = data1[(data1.BalancerType == "Smote")                & (data1.Setting == "Minimum inputs") & (data1.ClassifierType == "SVM")]
C = data1[(data1.BalancerType == "Smote")                & (data1.Setting == "Complete")       & (data1.ClassifierType == "SVM")]
D = data1[(data1.BalancerType == "Imbalanced")           & (data1.Setting == "No filtering")   & (data1.ClassifierType == "Tree")]
E = data1[(data1.BalancerType == "Smote")                & (data1.Setting == "Minimum inputs") & (data1.ClassifierType == "Tree")]
F = data1[(data1.BalancerType == "Imbalanced")           & (data1.Setting == "Minimum inputs") & (data1.ClassifierType == "SVM")]
LineAccuracy = [A.Accuracy.values[0], B.Accuracy.values[0], C.Accuracy.values[0], D.Accuracy.values[0], E.Accuracy.values[0], F.Accuracy.values[0]]
LineFmeasure = [A.Fmeasure.values[0], B.Fmeasure.values[0], C.Fmeasure.values[0], D.Fmeasure.values[0], E.Fmeasure.values[0], F.Fmeasure.values[0]]

paretoFrontColor=px.colors.qualitative.Plotly[0]

fig = go.Figure()

# Linea que une el frente de Pareto
fig.add_trace(
    go.Scatter(
        x=LineAccuracy,
        y=LineFmeasure,
        mode='lines',
        line={
            'width': 3,
            'color': paretoFrontColor,
        },
    )
)

# Puntos
for i in range(len(data1)):
    markerSel   = SetMarkerForm(data1.BalancerType[i])
    colorSel    = SetMarkerColor(data1.Setting[i])
    borderColor = SetBorderMarkerColor(data1.ClassifierType[i])

    fig.add_trace(
        go.Scatter(
            x=[data1.Accuracy.values[i]],
            y=[data1.Fmeasure.values[i]],
            mode='markers',
            marker={
                'symbol': markerSel,
                'color' : colorSel,
                'size'  : 10,
                'line'  : {
                    'width': 1.5,
                    'color': borderColor,
                },
            },
        )
    )

fig.update_layout(
    legend=dict(
        traceorder="normal",
        font=dict(
            size=14,
            color="black"
        ),
        bordercolor="Black",
        borderwidth=1,
        xanchor = "center", # Para centrarla horizontalmente
        x = 0.5,            # Para centrarla horizontalmente
        orientation="h",    # Para centrarla horizontalmente
    ),
    #autosize=False,
    width=750,
    height=750,
    xaxis_title="Accuracy",
    yaxis_title="F-measure",
    xaxis= {
        'linecolor': 'black',
        'linewidth': 1,
        'mirror': True,
        'titlefont': {
            'size':20,
            'color':'rgb(107, 107, 107)'
        },
        'tickfont': {
            'size':16,
            'color':'rgb(107, 107, 107)'
        },
        'tick0':      0,
        'dtick':      5,
        #"range":      [ 23, 75 ],
    },
    yaxis= {
        'linecolor': 'black',
        'linewidth': 1,
        'mirror': True,
        'titlefont': {
            'size':20,
            'color':'rgb(107, 107, 107)'
        },
        'tickfont': {
            'size':16,
            'color':'rgb(107, 107, 107)'
        },
        'tick0':      25,
        'dtick':      5,
        #"range":      [ -2, 85 ],
    },
    margin={
        'l':   10,
        'r':   10,
        'b':   10,
        't':   10,
        'pad': 2
    },
    
)

fig.show() # Para que salga el resultado

#### Se guarda el diagrama ####
pio.write_image(fig, "figura.svg")